In [ ]:
import pandas as pd
import numpy as np

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

load one of the saved programs in *ml/pipeline/programs*

In [ ]:
program = factories.Program.factory('2020-05-25', version='1.0')

predict on validation dataset

In [ ]:
program.predict(dataset_params = {
    'from': '2020-01-01',
    'to': '2020-02-01'
}, locked=True)

In [ ]:
program.bet()

In [ ]:
program.bets.profit.describe()